In [19]:
from packages import *
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import Dropout

In [17]:
model = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg')

In [18]:
class BaselineModel(tf.keras.Model):
    def __init__(self):
        super(BaselineModel, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        
        self.inception_v4 = InceptionResNetV2(include_top=False, weights='imagenet', pooling='avg', )

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x1 = self.conv1_1(input_tensor)
        x2 = self.conv1_2(input_tensor)
        x3 = self.conv1_3(input_tensor)
        x1 = self.pool(x1)
        x2 = self.pool(x2)
        x3 = self.pool(x3)
        x1 = self.conv2_1(x1)
        x2 = self.conv2_2(x2)
        x3 = self.conv2_3(x3)
        x1 = self.pool(x1)
        x2 = self.pool(x2)
        x3 = self.pool(x3)
        x1 = self.flatten(x1)
        x2 = self.flatten(x2)
        x3 = self.flatten(x3)
        x = self.concat([x1, x2, x3])
        x = self.fc1(x)
        x = self.fc2(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        
        return x

device = '/device:GPU:0'   # Change this to a CPU/GPU as you wish!
# device = '/cpu:0'        # Change this to a CPU/GPU as you wish!
print_every = 700
num_epochs = 10

input_shape = (None, 32, 32, 3)
channel_1 = 64
channel_2 = 32
num_classes = 10
hidden_dim = 50
# model = CustomConvNet(channel_1, channel_2, hidden_dim, num_classes)
# model.build(input_shape)

# model.summary()
# tf.keras.utils.plot_model(model)
def model_init_fn():
    return CustomConvNet(channel_1, channel_2, hidden_dim, num_classes)

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_203 (Bat (None, None, None, 3 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, None, None, 3 0           batch_normalization_v1_203[0][0] 
________________________________________________________________________________